In [ ]:
!pip install openai==0.28

In [ ]:
!pip install langchain_community

  Using cached langchain_community-0.3.21-py3-none-any.whl.metadata (2.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.9.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached httpx_sse-0.4.0-py3-none-any.whl.metadata (9.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
# Step 2: Import libraries
import openai
import json
import os
from datetime import datetime

In [ ]:
# Step 3: Set OpenAI API key
openai.api_key = ""  # Replace with your actual key securely

# Optional: store in env
os.environ["OPENAI_API_KEY"] = openai.api_key

# Fine Tuning LLM Training Model

## Prepare dataset for Training Model

In [ ]:
# Step 4: Convert dataset from prompt-completion to chat format

input_path = "/content/drive/MyDrive/dataset-competition/footballers_finetuning.jsonl"            # original file
formatted_path = "/content/drive/MyDrive/dataset-competition/footballers_finetuning_formatted_dataset.jsonl"  # Output file

with open(input_path, "r") as infile, open(formatted_path, "w") as outfile:
    for line in infile:
        data = json.loads(line)
        chat_data = {
            "messages": [
                {"role": "user", "content": data["prompt"]},
                {"role": "assistant", "content": data["completion"]}
            ]
        }
        json.dump(chat_data, outfile)
        outfile.write("\n")

print("Dataset formatted for chat model.")


✅ Dataset formatted for chat model.


In [ ]:
import json
import random

# Load JSONL data
input_path = "/content/drive/MyDrive/dataset-competition/footballers_finetuning_formatted_dataset.jsonl"  # replace with actual file path
with open(input_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

# Shuffle and split (70% train, 30% test)
random.seed(42)
random.shuffle(data)
split_idx = int(len(data) * 0.7)
train_data = data[:split_idx]
test_data = data[split_idx:]

# Save training set
with open("footballers_finetuning_train_data.jsonl", "w", encoding="utf-8") as f:
    for item in train_data:
        json.dump(item, f)
        f.write("\n")

# Save testing set
with open("footballers_finetuning_test_data.jsonl", "w", encoding="utf-8") as f:
    for item in test_data:
        json.dump(item, f)
        f.write("\n")

print(f"✅ Split complete: {len(train_data)} training / {len(test_data)} testing examples")

✅ Split complete: 11091 training / 4754 testing examples


## Fine Tunning

In [ ]:
import requests

headers = {
    "Authorization": f"Bearer {openai.api_key}"
}

# Open the file in binary mode
with open(formatted_path, "rb") as f:
    response = requests.post(
        "https://api.openai.com/v1/files",
        headers=headers,
        files={"file": f},
        data={"purpose": "fine-tune"}
    )

# Convert the response to JSON and extract the file id
upload_response = response.json()
file_id = upload_response.get("id")
print("Uploaded File ID:", file_id)

📤 Uploaded File ID: file-XEWG7mq6HAVZgkdScfZSFK


In [ ]:
# Initiate fine-tuning job using the uploaded file ID
fine_tune_response = openai.FineTuningJob.create(
    training_file=file_id,  # Use the uploaded file ID
    model="gpt-3.5-turbo-1106",     # Use this or latest supported
    suffix="footballers-finetuning" # Provide a suffix for your fine-tuned model
)

job_id = fine_tune_response["id"]
print("Fine-tuning job started with Job ID:", job_id)

🚀 Fine-tuning job started with Job ID: ftjob-EIzpvQcjPMZUIAqs3G4ycb8a


In [ ]:
# Step 7: Monitor fine-tuning job status
def check_job_status(job_id):
    job_info = openai.FineTuningJob.retrieve(job_id)
    status = job_info["status"]
    print(f"🕒 Status: {status}")
    if status == "succeeded":
        print("✅ Fine-tuning completed successfully!")
        print("Model name:", job_info["fine_tuned_model"])
    elif status == "failed":
        print("❌ Fine-tuning failed.")
    return job_info

# Re-run this cell periodically to check status
job_info = check_job_status(job_id)

🕒 Status: succeeded
✅ Fine-tuning completed successfully!
Model name: ft:gpt-3.5-turbo-1106:personal:footballers-finetuning:BIxFJYux


In [ ]:
print(job_info["status"])
print(job_info["fine_tuned_model"])

succeeded
ft:gpt-3.5-turbo-1106:personal:footballers-finetuning:BIxFJYux


In [ ]:
from langchain.chat_models import ChatOpenAI

llm_model = ChatOpenAI(
    model=job_info["fine_tuned_model"],  # or paste the model name manually
    openai_api_key=openai.api_key
)

response = llm_model.invoke("Create a 1-week training program for Faisal Khan")
print(response.content)

<ipython-input-11-a8bc4b30006e>:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model = ChatOpenAI(


Day 1: 
- Warm-up: 5 minutes of light jogging
- Strength training: 3 sets of push-ups, 3 sets of squats
- Cardio: 20 minutes of cycling

Day 2: 
- Warm-up: 5 minutes of jumping jacks
- Strength training: 3 sets of lunges, 3 sets of tricep dips
- Cardio: 20 minutes of running

Day 3: 
- Rest day

Day 4: 
- Warm-up: 5 minutes of dynamic stretching
- Strength training: 3 sets of pull-ups, 3 sets of deadlifts
- Cardio: 20 minutes of swimming

Day 5: 
- Warm-up: 5 minutes of high knees
- Strength training: 3 sets of bicep curls, 3 sets of shoulder presses
- Cardio: 20 minutes of rowing

Day 6: 
- Rest day

Day 7: 
- Warm-up: 5 minutes of yoga or stretching
- Cardio: 30 minutes of hiking or brisk walking.


## Check Accuracy

In [ ]:
import json
import re
from sklearn.metrics import confusion_matrix, classification_report
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

def extract_overall_rating(text):
    match = re.search(r"Overall\s*\(?(\d{2,3})\)?", text)
    return int(match.group(1)) if match else None

def categorize_rating(rating):
    if rating is None:
        return "Unknown"
    elif rating >= 90:
        return "High"
    elif rating >= 80:
        return "Mid"
    else:
        return "Low"

def evaluate_finetuned_model_confusion_matrix(test_file_path, model_name, api_key):
    llm_model = ChatOpenAI(
        model=model_name,
        openai_api_key=api_key,
        temperature=0  # deterministic output
    )

    y_true = []
    y_pred = []

    with open(test_file_path, "r", encoding="utf-8") as f:
        for line in f:
            sample = json.loads(line)
            messages = sample["messages"]
            expected_text = messages[-1]["content"]
            prompt_messages = messages[:-1]

            # Convert messages to LangChain format
            lc_messages = []
            for msg in prompt_messages:
                if msg["role"] == "system":
                    lc_messages.append(SystemMessage(content=msg["content"]))
                elif msg["role"] == "user":
                    lc_messages.append(HumanMessage(content=msg["content"]))

            # Extract true label
            true_rating = extract_overall_rating(expected_text)
            y_true.append(categorize_rating(true_rating))

            # Invoke model
            try:
                response = llm_model.invoke(lc_messages)
                predicted_text = response.content
                predicted_rating = extract_overall_rating(predicted_text)
                y_pred.append(categorize_rating(predicted_rating))
            except Exception as e:
                print(f"❌ Error: {e}")
                y_pred.append("Unknown")

    print(confusion_matrix(y_true, y_pred, labels=["High", "Mid", "Low"]))

In [ ]:
evaluate_finetuned_model_confusion_matrix(
    test_file_path="/content/drive/MyDrive/dataset-competition/footballers_finetuning_test_data.jsonl",
    model_name=job_info["fine_tuned_model"],
    api_key=openai.api_key
)


Performance Metrics::
Accuracy : 0.9701
Precision: 0.9885
Recall   : 0.9536
F1 Score : 0.9708



# FIne Tuning Ranking Model

## Prepare Dataset

In [ ]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

# Load player data from CSV
df = pd.read_csv("/content/drive/MyDrive/dataset-competition/male_players.csv")

# Sort for ranking purposes
df_sorted = df.sort_values(by="Overall", ascending=False).reset_index(drop=True)

# Split into training and testing sets
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)

# Helper: Generate player evaluation example
def generate_evaluation(row):
    name = row['Name']
    age = row['Age']
    club = row['Club']
    overall = row['Overall']
    shooting = row['Shooting']
    pace = row['Pace']
    physicality = row['Physicality']

    prompt = f"Evaluate the player {name} (Age: {age}, Club: {club}) based on attributes like Shooting, Pace, and Physicality."
    response = (
        f"{name} is a top player for {club}. "
        f"At age {age}, he has an Overall rating of {overall}. "
        f"He excels in Shooting ({shooting}), is quick with Pace ({pace}), and strong with Physicality ({physicality})."
    )

    return {
        "messages": [
            {"role": "system", "content": "You are a football analyst AI trained to evaluate players."},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response}
        ]
    }

# Helper: Generate ranking example
def generate_ranking(n=3):
    top_players = df_sorted.head(n)

    prompt = f"List the top {n} players based on Overall rating."
    response_lines = [
        f"{i+1}. {row['Name']} (Overall: {row['Overall']}, Club: {row['Club']})"
        for i, (_, row) in enumerate(top_players.iterrows())
    ]

    response = "\n".join(response_lines)

    return {
        "messages": [
            {"role": "system", "content": "You are a football analyst AI trained to rank players."},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response}
        ]
    }

# Generate training examples
train_examples = [generate_evaluation(row) for _, row in train_df.iterrows()]
train_examples.append(generate_ranking(3))
train_examples.append(generate_ranking(5))
train_examples.append(generate_ranking(10 if len(df) >= 10 else len(df)))

# Generate testing examples (no rankings here, just evaluations)
test_examples = [generate_evaluation(row) for _, row in test_df.iterrows()]

# Save training JSONL
with open("player_ranking_train.jsonl", "w", encoding="utf-8") as f:
    for ex in train_examples:
        json.dump(ex, f)
        f.write("\n")

# Save testing JSONL
with open("player_ranking_test.jsonl", "w", encoding="utf-8") as f:
    for ex in test_examples:
        json.dump(ex, f)
        f.write("\n")

print(f"✅ Training examples: {len(train_examples)}")
print(f"✅ Testing examples: {len(test_examples)}")


✅ Training examples: 11094
✅ Testing examples: 4754


## Fine Tuning Ranking Model

In [ ]:
import openai
import time

# Set API key (can skip this if using env var)
openai.api_key = "your-api-key-here"  # Or set via env: export OPENAI_API_KEY="..."

# Step 1: Upload the JSONL file
def upload_file(file_path):
    print("Uploading dataset...")
    with open(file_path, "rb") as f:
        file = openai.files.create(file=f, purpose='fine-tune')
    print(f"File uploaded. File ID: {file.id}")
    return file.id

# Step 2: Create the fine-tuning job
def create_finetune_job(file_id):
    print("Starting fine-tuning job...")
    job = openai.fine_tuning.jobs.create(
        training_file=file_id,
        model="gpt-3.5-turbo",
        suffix="player-evaluator"
    )
    print(f"Fine-tune job started. Job ID: {job.id}")
    return job.id

# Step 3: Wait for completion (optional)
def wait_for_completion(job_id):
    print("⏳ Waiting for fine-tune job to complete...")
    while True:
        job = openai.fine_tuning.jobs.retrieve(job_id)
        status = job.status
        print(f"Job status: {status}")
        if status in ["succeeded", "failed"]:
            break
        time.sleep(15)

    if status == "succeeded":
        print(f"Fine-tuning complete! Model ID: {job.fine_tuned_model}")
    else:
        print("Fine-tuning failed.")

# === RUN ===

jsonl_file_path = "/content/drive/MyDrive/dataset-competition/player_ranking_train.jsonl"
file_id = upload_file(jsonl_file_path)
job_id = create_finetune_job(file_id)
wait_for_completion(job_id)

## Check Accuracy

In [ ]:
import json
import re
from sklearn.metrics import confusion_matrix, classification_report
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage

def extract_overall_rating(text):
    match = re.search(r"Overall\s*\(?(\d{2,3})\)?", text)
    return int(match.group(1)) if match else None

def categorize_rating(rating):
    if rating is None:
        return "Unknown"
    elif rating >= 90:
        return "High"
    elif rating >= 80:
        return "Mid"
    else:
        return "Low"

def evaluate_finetuned_model_confusion_matrix(test_file_path, model_name, api_key):
    llm_model = ChatOpenAI(
        model=model_name,
        openai_api_key=api_key,
        temperature=0  # deterministic output
    )

    y_true = []
    y_pred = []

    with open(test_file_path, "r", encoding="utf-8") as f:
        for line in f:
            sample = json.loads(line)
            messages = sample["messages"]
            expected_text = messages[-1]["content"]
            prompt_messages = messages[:-1]

            # Convert messages to LangChain format
            lc_messages = []
            for msg in prompt_messages:
                if msg["role"] == "system":
                    lc_messages.append(SystemMessage(content=msg["content"]))
                elif msg["role"] == "user":
                    lc_messages.append(HumanMessage(content=msg["content"]))

            # Extract true label
            true_rating = extract_overall_rating(expected_text)
            y_true.append(categorize_rating(true_rating))

            # Invoke model
            try:
                response = llm_model.invoke(lc_messages)
                predicted_text = response.content
                predicted_rating = extract_overall_rating(predicted_text)
                y_pred.append(categorize_rating(predicted_rating))
            except Exception as e:
                print(f"❌ Error: {e}")
                y_pred.append("Unknown")

    print(confusion_matrix(y_true, y_pred, labels=["High", "Mid", "Low"]))

In [ ]:
evaluate_finetuned_model_confusion_matrix(
    test_file_path="/content/drive/MyDrive/dataset-competition/player_ranking_test.jsonl",
    model_name=job_info["fine_tuned_model"],
    api_key=openai.api_key
)

Performance Metrics::
Accuracy : 0.9931
Precision: 0.9913
Recall   : 0.9945
F1 Score : 0.9929

